### Before running the code cell below, take a look at variables. Change the variable KERNEL_DIR to directory containing jupyter kernels.

##### It will make directory from where you run it, for installing pip packages and to make it isolated env.

In [ ]:
%%bash
#!/bin/bash

# -------------- Config ----------------
LCG_VER="LCG_96python3"
LCG_ARCH="x86_64-centos7-gcc8-opt"
KERNEL_NAME="LCG_96_CENTOS7_GCC8"

MAKE_PY_KERNEL=true
PYTHON_KERNEL_NAME="Python3_${KERNEL_NAME}"
PYTHON_KERNEL_DISPLAY_NAME="Python 3 ${KERNEL_NAME}"

MAKE_CPP_KERNEL=false
CPP_KERNEL_NAME="CPP_${KERNEL_NAME}"
CPP_KERNEL_DISPLAY_NAME="C++ ${KERNEL_NAME}"

KERNELS_DIR="!!!! PATH TO YOUR !!!/share/jupyter/kernels"
# -------------------------------------

# making directories for keeping wrapper
# and making directory names such that
# it can be used for LCG setup.

set -e
# LCG
mkdir -p $LCG_VER
mkdir -p $LCG_VER/$LCG_ARCH
cd $LCG_VER/$LCG_ARCH
mkdir -p bin

# Variables,
# DIR -> since we will be using kernel (within notebook) in different locations,
#       we first get full path of where wrapper is,
#       we go there and setup kernel everytime we start/restart kernel.
#
# we get LCG versiona and ARCH from directories we made.
# 
# added PYTHON USER BASE for installing extra packages from pip and,
# keeping it isolated for this kernel.
# 
# pre-appending PYTHON USER BASE to PYTHON PATH,
# so that pip installed are loaded first
# 
# cd - : After everything is done, we go back and execute python

# Make a wrapper script to load LCG python
cat <<'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
cd $DIR
LCG_ARCH=$( cd ../; basename $(pwd) )
LCG_VER=$( cd ../../; basename $(pwd) )
source /cvmfs/sft.cern.ch/lcg/views/setupViews.sh $LCG_VER $LCG_ARCH
export PYTHONUSERBASE=`cd ../;pwd`
PY_VER=`python -c "import sys; print('python{0}.{1}'.format(*sys.version_info))"`
export PYTHONPATH=$PYTHONUSERBASE/lib/$PY_VER/site-packages:$PYTHONPATH
export PATH=$PYTHONUSERBASE/bin:$PATH
cd -
exec python3 "$@"
EOF

chmod +x bin/python_wrapper.sh

if [ "$MAKE_PY_KERNEL" = true ] ; then
    # Create the kernel python
    mkdir -p "$KERNELS_DIR/$PYTHON_KERNEL_NAME"
    cat <<EOF > "$KERNELS_DIR/$PYTHON_KERNEL_NAME/kernel.json"
    {
     "display_name": "$PYTHON_KERNEL_DISPLAY_NAME", 
     "language": "python", 
     "argv": [
      "$PWD/bin/python_wrapper.sh", 
      "-m", 
      "ipykernel_launcher", 
      "-f", 
      "{connection_file}"
     ]
    }
EOF
fi

if [ "$MAKE_CPP_KERNEL" = true ] ; then
    # Create the kernel ROOT C++
    mkdir -p "$KERNELS_DIR/$CPP_KERNEL_NAME"
    cat <<EOF > "$KERNELS_DIR/$CPP_KERNEL_NAME/kernel.json"
    {
     "display_name": "$CPP_KERNEL_DISPLAY_NAME", 
     "language": "c++", 
     "argv": [
      "$PWD/bin/python_wrapper.sh", 
      "-m", 
      "JupyROOT.kernel.rootkernel", 
      "-f", 
      "{connection_file}"
     ]
    }
EOF
fi

#### Run the code below with newly installed kernel, change the cell type from 'Raw' to 'Code'.